In [1]:
import sys, os
import argparse

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../")))

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "")))

In [2]:
from fedml_api.data_preprocessing.cifar10.data_loader import load_partition_data_cifar10
from fedml_api.standalone.fedavg.my_model_trainer_classification import MyModelTrainer as MyModelTrainerCLS
from fedml_api.model.contrastive_cv.resnet_with_embedding import Resnet56
from CovaMNet import CovaMResnet56

import torch
from torch import nn
from collections import OrderedDict
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import random
import pickle

# from triplet_loss import TripletLoss
from hard_triplet_loss import TripletLoss

KeyboardInterrupt: 

In [ ]:
dataset = 'cifar10'
data_dir = "./../../../data/cifar10"
# partition_method = 'hetero'
partition_method = 'homo'
partition_alpha = 0.5
client_num_in_total = 3
batch_size = 100
total_epochs = 500
save_model_path = 'model/client_{0}_covaMNet_epochs_{1}.pt'

device = 'cuda:0'

In [ ]:
with open(f'dataset_{partition_method}_{client_num_in_total}.pickle', 'rb') as f:
    dataset = pickle.load(f)
# [train_data_num, test_data_num, train_data_global, test_data_global, \
# #             train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
# #             class_num]

In [ ]:
model = Resnet56(class_num=dataset[-1], neck='bnneck')
model.load_state_dict(torch.load(str.format('model/cs_{0}_{1}_client_{2}_oral_epochs_{3}.pt', client_num_in_total, partition_method, 0, 200)))

In [ ]:
def test(model, test_data, device):
    model.to(device)
    model.eval()

    metrics = {
        'test_correct': 0,
        'test_loss': 0,
        'test_total': 0
    }

    criterion = nn.CrossEntropyLoss().to(device)

    with torch.no_grad():
        for batch_idx, (x, target) in enumerate(test_data):
            x = x.to(device)
            target = target.to(device)
            pred, feat = model(x)
            loss = criterion(pred, target)

            _, predicted = torch.max(pred, -1)
            correct = predicted.eq(target).sum()

            metrics['test_correct'] += correct.item()
            metrics['test_loss'] += loss.item() * target.size(0)
            metrics['test_total'] += target.size(0)
            
    return metrics

In [ ]:
# caculate covariance
# def cal_covariance(input):

# #     CovaMatrix_list = []
# #     for i in range(len(input)):
#     support_set_sam = input
#     B, C = support_set_sam.size()
# #         B, C, h, w = support_set_sam.size()

#     support_set_sam = support_set_sam.permute(1, 0)
#     support_set_sam = support_set_sam.contiguous().view(C, -1)
# #         mean_support = torch.mean(support_set_sam, 0, True)
#     mean_support = torch.mean(support_set_sam, 1, True)
#     support_set_sam = support_set_sam-mean_support

#     covariance_matrix = support_set_sam@torch.transpose(support_set_sam, 0, 1)
# #         covariance_matrix = support_set_sam@support_set_sam.t()
# #         covariance_matrix = torch.div(covariance_matrix, h*w*B-1)
#     covariance_matrix = torch.div(covariance_matrix, B-1)
# #     CovaMatrix_list.append(covariance_matrix)
#     return covariance_matrix

# #     return CovaMatrix_list
def cal_covariance(input):

    CovaMatrix_list = []
    mean_list = []
    for i in range(len(input)):
        support_set_sam = input[i]
        support_set_sam = torch.unsqueeze(support_set_sam, 0)
        B, C, h, w = support_set_sam.size()
        

        support_set_sam = support_set_sam.permute(1, 0, 2, 3)
        support_set_sam = support_set_sam.contiguous().view(C, -1)
        
#         query_sam_norm = torch.norm(support_set_sam, 2, 1, True) 
#         if query_sam_norm.min() == 0:
#             print('query_sam_norm.min() == 0')
#             print(torch.count_nonzero(support_set_sam))

        mean_support = torch.mean(support_set_sam, 1, True)
        mean_list.append(mean_support)

        support_set_sam = support_set_sam-mean_support

        covariance_matrix = support_set_sam@torch.transpose(support_set_sam, 0, 1)
        covariance_matrix = torch.div(covariance_matrix, h*w*B-1)
        CovaMatrix_list.append(covariance_matrix)

#     return covariance_matrix
    return CovaMatrix_list, mean_list

# def cal_mean(input):
#     means_list = []
#     for in in range(len(input)):
        
        

cl = [torch.zeros((256,256)) for i in range(10)]
ml = [torch.zeros((256,64)) for i in range(10)]
lbd = 0.999
def extract_features(model, data_loader, device):
    model.to(device)
    model.eval()
    
    features = []
    labels = []
    
    with torch.no_grad():
        for batch_idx, (x, l) in enumerate(data_loader):
            x, l = x.to(device), l.to(device)
            
            score, feats = model(x)
#             B, C = feats.size()
#             feats = feats.permute(1, 0)
#             print(feats.shape)
#             feats = feats.contiguous().view(C, -1)
#             mean_support = torch.mean(feats, 1, True)
            
#             feats = feats-mean_support
#             covariance_matrix = feats@torch.transpose(feats, 0, 1)
#             covariance_matrix = torch.div(covariance_matrix, B-1)
#             print(covariance_matrix.shape)
            
            
#             break
#             f = feats[0]
#             print(f.size())
#             f = f.contiguous().view(256, -1)
#             mean_f = torch.mean(f)
#             print(mean_f)
#             f = f - mean_f
#             print(f.shape)
#             covaM = f@f.t()
#             print(covaM.size())
            
            # B
#             covaM_list = []
#             print(feats[0].max())
#             print(feats[0].min())
            covaM_list, mean_list = cal_covariance(feats)
#             print(covaM_list[0].max())
#             print(covaM_list[0].min())
#             break
            
            
#             print(covaM_list[0])
#             print(covaMs)
#             print(covaMs - torch.transpose(covaMs, 0, 1))
#             break
            for covaM, f, label in zip(covaM_list, feats, l):
#                 a = covaM - torch.transpose(covaM, 0, 1)
#                 print(a.sum())
                for i in range(len(cl)):
#                     print(f'{label.data.cpu()} == {i}')
                    if label.data.cpu() == i:
#                         print(f'{label.data.cpu()} == {i}')
#                         covaM_list[i] = covaM_list[i] + covaM
                        cl[i] = lbd * cl[i] + (1-lbd) * covaM.cpu()
                        f = torch.unsqueeze(f, 0)
                        B, C, h, w = f.size()

                        f = f.permute(1, 0, 2, 3)
                        f = f.contiguous().view(C, -1)
                        ml[i] = lbd * ml[i] + (1-lbd) * f.cpu()
#             for j in range(len(covaMs)):
#                 for i in range(len(covaM_list)):
#                     if l[j].data.cpu() == i:
#                         covaM_list[i] = lbd * covaM_list[i] + (1-lbd) * covaMs[j]
#             print(feats.shape)
#             for feat, label in zip(feats, l):
#                 features.append(feat.cpu())
#                 labels.append(label.cpu())
#     return 

# metrics = test(model, dataset[3], device)
# test_correct = metrics['test_correct']/metrics['test_total']
# test_loss = metrics['test_loss']/metrics['test_total']
# print(metrics['test_total'])
# print(f'test_correct: {test_correct}; test_loss: {test_loss}')

extract_features(model, dataset[2], device)

In [ ]:
# print(cl[0])
for i in range(len(cl)):
    cl[i] = cl[i].to(device)
    print(cl[i].sum())
    print(cl[i].min())
    print(cl[i].max())
#     print(cl[i])
# print(device)
print(cl[1])
print(cl[0] - torch.transpose(cl[0], 0, 1))
print(ml[1])

In [ ]:
def test_model(model, test_data, device):
    model.to(device)
    model.eval()

    metrics = {
        'test_correct': 0,
        'test_loss': 0,
        'test_total': 0
    }

    criterion = nn.CrossEntropyLoss().to(device)

    with torch.no_grad():
        for batch_idx, (x, target) in enumerate(test_data):
            x = x.to(device)
            target = target.to(device)
            pred, feat = model(x, cl, support_ml)
            loss = criterion(pred, target)

            _, predicted = torch.max(pred, -1)
            correct = predicted.eq(target).sum()

            metrics['test_correct'] += correct.item()
            metrics['test_loss'] += loss.item() * target.size(0)
            metrics['test_total'] += target.size(0)
            
    return metrics


def train_model(client, epochs):
#     learning_rate = 0.001
#     wd = 0.0001
#     learning_rate = 0.00035
#     wd = 0.0005
    
    margin = 0.3
    
    client.model.to(client.device)
    client.model.train()
    
#     triplet = TripletLoss(margin)
#     triplet = HardTripletLoss(margin)
    criterion = nn.CrossEntropyLoss().to(device)
    
#     curr_lr = learning_rate
#     optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, client.model.parameters()), lr=curr_lr,
#                                          weight_decay=wd, amsgrad=True)
    optimizer = torch.optim.SGD(client.model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

    epoch_loss = []
    total_step = len(client.train_data)
    for epoch in range(epochs):
        client.model.train()
        batch_loss = []
        for batch_idx, (x, labels) in enumerate(client.train_data):
            x, labels = x.to(device), labels.to(device)
#             print(labels.shape)
            client.model.zero_grad()
            support_ml = []
            for l in labels:
                support_ml.append(ml[l.data].to(device))
            score, feat = client.model(x, cl, support_ml)
            
#             score = client.model(x)
            
#             dist_ap, relative_p_inds = torch.max(dist_mat[is_pos].contiguous().view(-1), dim=1, keepdim=True)
#             print(dist_mat[is_pos].contiguous())
#             dist_ap, relative_p_inds = torch.max(
#                 dist_mat[is_pos].contiguous().view(N, -1), 1, keepdim=True)
            
#             tri_loss, tri_prec = triplet(feat, labels)
#             loss = criterion(score, labels) + tri_loss
            loss = criterion(score, labels)
#             if epoch > 0:
#                 print(f'score: {score}')
#                 print(f'loss: {loss}')
#             if torch.gt(loss.data, 5):
#                 print(f'loss > 10; {loss}')
#             print(loss.data<100)
#             if loss.data > 100:
#                 print(f'loss>100; {loss}')
            loss.backward()
        
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            batch_loss.append(loss.item())
            
        
        
            
        if epoch % 100 == 0:
            torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epoch))
        epoch_loss.append(sum(batch_loss) / len(batch_loss))
        scheduler.step()
        print('Client Index = {}\tEpoch: {}\tLoss: {:.6f}'.format(
            client.id, epoch, sum(batch_loss) / len(batch_loss)))
        
        metrics = test(client.model, client.test_local, client.device)
        test_correct = metrics['test_correct']/metrics['test_total']
        test_loss = metrics['test_loss']/metrics['test_total']
        print(metrics['test_total'])
        print(f'test_correct: {test_correct}; test_loss: {test_loss}')
        
    torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epochs))

In [ ]:
class Client(object):
    def __init__(self, client_index, train_data_local_dict, train_data_local_num_dict, test_data_local_dict, device, model):
        self.id = client_index
        self.train_data = train_data_local_dict[self.id]
        self.local_sample_number = train_data_local_num_dict[self.id]
        self.test_local = test_data_local_dict[self.id]
        
        self.device = device
        self.model = model
model1 = CovaMResnet56(class_num=dataset[-1], neck='bnneck')
client_1 = Client(0, dataset[5], dataset[4], dataset[6], device, model1)

In [ ]:
# for name, module in model1.named_modules():
#     print(name)
# for name, module in model.named_modules():
#     print(name)
md = model.named_modules()
modules_dict = {k:v for k,v in md}
# print(modules_dict)
for name, module in model1.named_modules():
    if name == 'bottleneck':
        module.load_state_dict(modules_dict['bottleneck'].state_dict())
    if name == 'ce_classifier':
        module.load_state_dict(modules_dict['classifier'].state_dict())
#         module = copy.deepcopy(modules_dict['bottleneck'])


In [ ]:
train_model(client_1, 200)

In [ ]:
# client_1.model.state_dict()['bottleneck.weight']
for name, module in model.named_modules():
    print(name)

for name, module in client_1.model.named_modules():
#     if name == 'bottleneck':
#         for param_tensor in module.state_dict():
#             print(param_tensor)
#             print(model.state_dict()[f'bottleneck.{param_tensor}'] - module.state_dict()[param_tensor])
    if name == 'ce_classifier':
        for param_tensor in module.state_dict():
            print(param_tensor)
            print(model.state_dict()[f'classifier.{param_tensor}'] - module.state_dict()[param_tensor])
#             print(module.state_dict()[param_tensor])
# #         print(client1.model.state_dict()[''])
#         for param_tensor in client1.model.state_dict()['']
# for param_tensor in client_1.model.state_dict():
#     print(param_tensor)